In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-data-set/Day 3 - Semi-Focussed or Drowsy.csv
/kaggle/input/eeg-data-set/Day 3 - Unfocussed or Sleeping.csv
/kaggle/input/eeg-data-set/Day 3 - Focussed.csv
/kaggle/input/eeg-dataset-samples/Day 5 - Unfocussed or sleeping.csv
/kaggle/input/eeg-dataset-samples/Day 5 - Semi-Focussed or Drowsy.csv
/kaggle/input/eeg-dataset-samples/Day 4 - Semi-Focussed or Drowsy.csv
/kaggle/input/eeg-dataset-samples/Day 4 - Focussed.csv
/kaggle/input/eeg-dataset-samples/Day 6 - Focussed.csv
/kaggle/input/eeg-dataset-samples/Day 5 - Focussed.csv
/kaggle/input/eeg-dataset-samples/Day 3 - Focussed.csv
/kaggle/input/eeg-dataset-samples/Day 4 - Unfocussed or Sleeping.csv
/kaggle/input/eeg-dataset-samples/Day 6 - Unfocussed or Sleeping.csv
/kaggle/input/eeg-dataset-samples/Day 6 - Semi-Focussed or Drowsy.csv


In [2]:
pip install scikeras[tensorflow]

Note: you may need to restart the kernel to use updated packages.


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.optimizers import adam_v2
from tensorflow.python.keras.metrics import CategoricalAccuracy
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.models import Model
from keras.utils import to_categorical

In [6]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

In [7]:
focussed_day1 = pd.read_csv("/kaggle/input/eeg-data-set/Day 3 - Focussed.csv")
focussed_day1 = focussed_day1.dropna(axis=0)
drowsy_day1 = pd.read_csv("/kaggle/input/eeg-data-set/Day 3 - Semi-Focussed or Drowsy.csv")
unfocussed_day1 = pd.read_csv("/kaggle/input/eeg-data-set/Day 3 - Unfocussed or Sleeping.csv")
drowsy_day1 = drowsy_day1.dropna(axis=0)
unfocussed_day1 = unfocussed_day1.dropna(axis=0)

focussed_day2 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 4 - Focussed.csv")
focussed_day2 = focussed_day2.dropna(axis=0)
drowsy_day2 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 4 - Semi-Focussed or Drowsy.csv")
unfocussed_day2 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 4 - Unfocussed or Sleeping.csv")
drowsy_day2 = drowsy_day2.dropna(axis=0)
unfocussed_day2 = unfocussed_day2.dropna(axis=0)

focussed_day3 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 5 - Focussed.csv")
focussed_day3 = focussed_day3.dropna(axis=0)
drowsy_day3 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 5 - Semi-Focussed or Drowsy.csv")
unfocussed_day3 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 5 - Unfocussed or sleeping.csv")
drowsy_day3 = drowsy_day3.dropna(axis=0)
unfocussed_day3 = unfocussed_day3.dropna(axis=0)

focussed_day4 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 6 - Focussed.csv")
focussed_day4 = focussed_day4.dropna(axis=0)
drowsy_day4 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 6 - Semi-Focussed or Drowsy.csv")
unfocussed_day4 = pd.read_csv("/kaggle/input/eeg-dataset-samples/Day 6 - Unfocussed or Sleeping.csv")
drowsy_day4 = drowsy_day4.dropna(axis=0)
unfocussed_day4 = unfocussed_day4.dropna(axis=0)


In [8]:
merged = pd.concat([focussed_day1, drowsy_day1, unfocussed_day1, focussed_day2, drowsy_day2, unfocussed_day2,
                   focussed_day3, drowsy_day3, unfocussed_day3, focussed_day4, drowsy_day4, unfocussed_day4])
#merged

In [9]:
test_features = ['F7', 'F3', 'P7', 'O1', 'O2', 'P8', 'AF4']
test_value_features = ['Class']

In [10]:
X_train1 = merged[test_features]
y_train1 = merged[test_value_features]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size=0.2, random_state=0, stratify=y_train1)

In [12]:
y_train_encoded = pd.get_dummies(y_train)
y_test_encoded = pd.get_dummies(y_test)

In [13]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_test)

In [14]:
early_stopping = EarlyStopping(
    min_delta=0.001, 
    patience=20, 
    restore_best_weights=True,
)

In [15]:
def elu(x, alpha=1.0):
    return tf.keras.layers.ELU(alpha=alpha)(x)

In [16]:
def neural_network_model():
    nn_model = Sequential([
    layers.Dense(1024, activation=elu, input_shape=[7]),  
    layers.Dropout(0.3),
    layers.Dense(1024, activation=elu),
    layers.Dropout(0.3),
    layers.Dense(1024, activation=elu),
    layers.Dropout(0.3),
    layers.Dense(1024, activation=elu),
    layers.Dropout(0.3),
    layers.Dense(1024, activation=elu),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
    ])
    nn_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'],
    )
    return nn_model
    

In [18]:
nn_model = neural_network_model

In [17]:
nn_predictions = nn_model.fit(X_train_scaled, y_train_encoded, batch_size=32768, epochs=1000, validation_data = (X_val_scaled, y_test_encoded))

NameError: name 'nn_model' is not defined

In [19]:
svm_model = SVC(kernel='rbf', C=2, probability=True, max_iter = 250)

In [20]:
y_train_2 = y_train['Class'].to_numpy()

In [ ]:
#svm_model.fit(X_train_scaled, y_train_2)

In [ ]:
#y_test_3 = y_test['Class'].to_numpy()

In [ ]:
#predicted_labels = svm_model.predict(X_val_scaled)

In [ ]:
#accuracy_svm = accuracy_score(y_test_3, predicted_labels)
#accuracy_svm

In [21]:
def joint_loss(neural_network_loss, svm_loss):
    # Define the weights for balancing the contributions of each loss
    neural_network_weight = 0.75
    svm_weight = 0.25
    # Calculate the joint loss
    joint_loss = neural_network_weight * neural_network_loss + svm_weight * svm_loss
    return joint_loss

In [ ]:
#batch_size = 16384
#batch_indices = np.random.choice(len(X_train_scaled), size=batch_size, replace=False)

In [ ]:
#X_train_batch = X_train_scaled[batch_indices]
#y_train_batch = y_train[batch_indices]

In [ ]:
#neural_network_loss = neural_network_model.train_on_batch(X_train_batch, y_train_batch)

In [ ]:
#last_layer_index = len(neural_network_model.layers) - 1
#feature_extractor_model = Model(inputs=neural_network_model.input, outputs=neural_network_model.layers[last_layer_index].output)

In [ ]:
#extracted_features = feature_extractor_model.predict(X_train_batch)

In [ ]:
#svm_loss = svm_model.fit(extracted_features, y_train_batch)

In [ ]:
#loss = joint_loss(neural_network_loss, svm_loss)

In [ ]:
# Update the neural network model
#neural_network_model.train_on_batch(X_train_batch, y_train_batch)
# Update the SVM model
#svm_model.fit(extracted_features_batch, y_train_batch)

In [22]:
# Initialize the logistic regression model
logistic_regression_model = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', multi_class='ovr')

In [23]:
num_epochs = 10

In [33]:
for epoch in range(num_epochs):
    
    batch_size = 16384
    batch_indices = np.random.choice(len(X_train_scaled), size=batch_size, replace=False)
    
    X_train_batch = X_train_scaled[batch_indices]
    y_train_batch = y_train_encoded[batch_indices]
    y_train_2_batch =  y_train_2[batch_indices]
    
    #neural_network_loss = neural_network_model.train_on_batch(X_train_batch, y_train_batch)
    neural_network_loss = nn_model.train_on_batch(X_train_batch, y_train_batch)
    
    #last_layer_index = len(neural_network_model.layers) - 1
    last_layer_index = len(nn_model.layers) - 1
    #feature_extractor_model = Model(inputs=neural_network_model.input, outputs=neural_network_model.layers[last_layer_index].output)
    feature_extractor_model = Model(inputs=nn_model.input, outputs=nn_model.layers[last_layer_index].output)
    
    extracted_features = feature_extractor_model.predict(X_train_batch)
    
    flattened_features = extracted_features.reshape(len(extracted_features), -1)
    
    svm_loss = svm_model.fit(flattened_features, y_train_2_batch)
    #svm_loss = svm_model.fit(X_train_batch, y_train_2_batch)
    
    loss = joint_loss(neural_network_loss, svm_loss)
    
    # Update the neural network model
    neural_network_model.train_on_batch(X_train_batch, y_train_batch)
    # Update the SVM model
    #svm_model.fit(extracted_features_batch, y_train_2_batch)
    svm_model.fit(flattened_features, y_train_2_batch)
    #svm_model.fit(X_train_batch, y_train_2_batch)
    
    #logistic_regression_model.fit(extracted_features, y_train_batch)
    logistic_regression_model.fit(flattened_features, y_train_batch)

KeyError: "None of [Int64Index([3679729, 3073723, 2269312, 2016014,  300709, 2310979, 3957053,\n            2929527, 3265767, 2611852,\n            ...\n            4546178,  356838, 1007008, 4157749,  445983, 1983523, 1182794,\n              87467, 2328073, 1558672],\n           dtype='int64', length=16384)] are in the [columns]"

In [29]:
extracted_features.shape

NameError: name 'extracted_features' is not defined

In [28]:
y_train_2_batch.shape

NameError: name 'y_train_2_batch' is not defined

In [ ]:
logistic_regression_accuracy = logistic_regression_model.score(features_test, y_test)

In [ ]:
print(f"Logistic Regression Accuracy: {logistic_regression_accuracy:.4f}")

In [ ]:
# Initialize the logistic regression model
#logistic_regression_model = LogisticRegression(...)

In [ ]:
# Train the logistic regression model
#logistic_regression_model.fit(extracted_features_train, y_train)

In [ ]:
# Make predictions using the logistic regression model
#history = logistic_regression_model.predict(extracted_features_test)